# Exploring the demonstration models

This notebook shows several standard ways to interact with the demonstrations stored into the exp/run directories. 

In [ ]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import cv2

import matplotlib.pyplot as plt
import random
import torch
import json

import pathlib
from demonstration import Demonstration, list_demos, select_demo

In [ ]:
experiment = "demonstration"
run = "freeform"

exp = Config().get_experiment(experiment, run)

### Read out all the demonstrations from a run

In [ ]:
# read out all the demonstrations from a run
#demos = [item.name for item in exp.data_dir().iterdir() if item.is_dir()]
demos = list_demos(exp)
print(demos)

In [ ]:
demo = select_demo(exp)

###  Read out all the pictures in the demonstration

In [ ]:
demo = Demonstration(exp, demo)
print(demo)
print(f"Cameras found: {demo.cameras}")
print(f"There are {demo.maxsteps} steps in this demonstration")
print(f"This demonstration was recorded by the following cameras: {demo.cameras}")

# pick and show a random image from a random camera
cam = random.choice(demo.cameras)
imgno = random.randint(0, demo.maxsteps)
sensor_readings, image = demo.get_image(imgno, camera=cam)

fig, ax = plt.subplots()
ax.imshow(image)


# Move all the figures to a video

In [ ]:
def move_to_video_per_camera(demo: Demonstration, cam, delete_img_files = False):
    """Move the content of a specific camera into video"""
    if cam in demo.exp["cameras"]:
        params = demo.exp["cameras"][cam]
    else:
        params = demo.exp["cameras"]["all"]
    video_path = demo.get_video_path()
    image_paths = []
    # Initialize video writer
    if not video_path.exists():
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(video_path, fourcc, params["fps"], (params["width"], params["height"]))
        for i in range(demo.maxsteps):
            img_path = demo.get_image_path(i, camera=cam)
            image_paths.append(img_path)
            frame = cv2.imread(str(img_path))
            out.write(frame)
        out.release()
    # if specified, delete the image files
    if delete_img_files:
        for img_path in image_paths:
            img_path.unlink()

In [ ]:
def move_to_video(demo: Demonstration, delete_img_files = False):
    """Moves to the video the content all all cameras"""
    for cam in demo.cameras:
        move_to_video_per_camera(demo, cam, delete_img_files)

In [ ]:
from sensorprocessing.sp_helper import load_capture_to_tensor

def get_image_from_video(demo: Demonstration, i, camera=None, cache=False):
    """Extracts an image from the video. 
    FIXME: this function opens the video file, seeks and closes it, so it should be very inefficient, we should store the open one in the demonstration instead
    If cache is False, the function closes the open file
    """
    if camera is None:
        camera = demo.cameras[0]    
    if camera not in demo.videocap:    
        video_path = demo.get_video_path(camera)
        cap = cv2.VideoCapture(video_path)
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        demo.videocap[camera] = cap
    else:
        cap = demo.videocap[camera]
    ret, frame = cap.read()
    if ret:
        # CV2 reads by default in BGR... 
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # cv2.imwrite(output_image, frame)    
        image_to_process, image_to_show = load_capture_to_tensor(frame, transform=None)
    else:
        print(f"Could not read frame {i}")
    if not cache:
        demo.videocap[camera].release()
        demo.videocap[camera] = None
    return image_to_process, image_to_show        

In [ ]:
imgno = random.randint(0, demo.maxsteps)
vid_tensor, vid_image = get_image_from_video(demo, imgno)

img_tensor, img_image = demo.get_image(imgno, camera=cam)

fig, [ax1, ax2, ax3] = plt.subplots(1,3)
ax1.imshow(image)
ax3.imshow(vid_image)
ax2.imshow(img_image)


In [ ]:
vid_tensor - img_tensor